# Langchain Walkthrough

Inspired by: https://langchain-ai.github.io/langgraph/agents/agents/

Setup the environment

In [ ]:
import dotenv
import os
import logfire
import json

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")


# configure logfire
logfire.configure(token=logfire_token)

# 2. Instrumenteer LangChain (automatisch loggen van chains, LLM calls, tools, etc.)
logfire.instrument()

1 Definieer een basis agent

In [ ]:
from langgraph.prebuilt import create_react_agent

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")

basic_agent = create_react_agent(
    model=model,
    tools=[],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
)

Stel een vraag aan de agent

In [ ]:
# Run the agent
result = basic_agent.invoke(
    {"messages": [{"role": "user", "content": "geef een defintie van een agent"}]}
)

print(result["messages"][-1].content)

Definieer tools voor de agent

In [4]:
tool_agent = create_react_agent(
    model=model,
    tools=[tf.calculate_btw, tf.calculate_discount, tf.get_inwoners_gemeente],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
)

config = {"configurable": {"thread_id": "1"}}

Test een aantal tool calls

In [ ]:
result2 = tool_agent.invoke(
    {
        "messages": {
            "role": "user",
            "content": "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld",
        }
    },
    config=config,
)
print(result2["messages"][-1].content)

In [ ]:
result = tool_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien.",
            }
        ]
    }
)

print(result["messages"][-1].content)

In [ ]:
result = tool_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan.",
            }
        ]
    }
)
print(result["messages"][-1].content)

Voeg geheugen toe aan de agent

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent

checkpoint = InMemorySaver()

mem_agent = create_react_agent(
    model=model,
    tools=[tf.calculate_btw, tf.calculate_discount, tf.get_inwoners_gemeente],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
    checkpointer=checkpoint,
)

config = {"configurable": {"thread_id": "1"}}

result = mem_agent.invoke(
    {
        "messages": {
            "role": "user",
            "content": "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld",
        }
    },
    config=config,
)
print(result["messages"][-1].content)

In [ ]:
result = mem_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
            }
        ]
    },
    config=config,
)
print(result["messages"][-1].content)